In [2]:
from sys import platform
import os
import cv2
import json
from pupil_apriltags import Detector
from glob import glob
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt


In [3]:
from math import pi
def position_error(T_e, T_gt):

    return np.linalg.norm(T_e- T_gt)


def rotation_error(R_e, R_gt):
    
    return (180/pi) * np.arccos((np.trace(np.matmul(R_e, R_gt.T))-1)/2)

In [4]:
if platform == "linux" or platform == "linux2":  
# linux
    data_path  = "/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1"
    data_pi_path = "/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1"
elif platform == "win32":
# Windows...
    data_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1"
    data_pi_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1"


    
DATA = "IPHONE" # "PI" or "IPHONE"

Evaluation = {}

at_detector = Detector(
        families="tagStandard41h12",
        nthreads=1,
        quad_decimate=1.0,
        quad_sigma=0.0,
        refine_edges=1,
        decode_sharpening=0.25,
        debug=0,
    )

if DATA == "IPHONE":

    images_path = os.path.join(data_path, "frames")
    pose_path = os.path.join(data_path, "pose")
    mesh_fullpath = os.path.join(data_path, "data3d/textured_output.obj")
    depth_path = os.path.join(data_path, "depth")

elif DATA == "PI":
    data_path = data_pi_path
    images_path = os.path.join(data_path, "images_undistorted")

tag_sizes = np.arange(0.01, 0.15, 0.01)


In [6]:
if DATA == "IPHONE":

    images_path = os.path.join(data_path, "frames")
    pose_path = os.path.join(data_path, "pose")
    mesh_fullpath = os.path.join(data_path, "data3d/textured_output.obj")
    depth_path = os.path.join(data_path, "depth")

elif DATA == "PI":
    data_path = data_pi_path
    images_path = os.path.join(data_path, "images_undistorted")

images_files = os.listdir(images_path)
images_files.sort()

visualization = {}

for i, image_file in enumerate(images_files):
    # if i%5 != 0:
    #     continue

    if len(visualization.keys()) == 6:
        break
    if DATA == "PI":
        img_id = image_file.split(".")[0]
        
    elif DATA == "IPHONE":
        img_id = image_file[6:11]

    img = cv2.imread(os.path.join(images_path, image_file), cv2.IMREAD_GRAYSCALE)
    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    if DATA == "IPHONE":
    
        camera_param_file = image_file.replace(".jpg", ".json")
        with open(os.path.join(pose_path, camera_param_file), 'r') as f:
            camera_param = json.load(f)

        intrinsics = np.array(camera_param["intrinsics"]).reshape(3, 3)

        projectionMatrix = np.array(camera_param["projectionMatrix"]).reshape(4, 4)

    elif DATA == "PI":
        intrinsics = np.array([[766.2927454396544, 0.0, 543.6272327745995],
                            [0.0, 766.3976103393867, 566.0580149497666],
                            [0.0, 0.0, 1.0]])

        projectionMatrix = np.eye(4)
        
    fxfycxcy= [intrinsics[0, 0], intrinsics[1, 1], intrinsics[0, 2], intrinsics[1, 2]]
    principal_point = np.array([intrinsics[0, 2], intrinsics[1, 2]])
    focal_length = (intrinsics[0, 0] + intrinsics[1, 1]) / 2    

    true_distances = []
    
    for tag_size in tag_sizes:   
            
        tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=tag_size)
        if len(tags == 0):
            break

        # Test with only one tag
        tag_test = tags[-1]
        tag_id = tag_test.tag_id

        if tag_id in visualization.keys():
            break
        
            
        tag_position = np.array(tag_test.pose_t)
        tag_cam_distance = np.linalg.norm(tag_position)
        true_distances.append(tag_cam_distance)

    true_distances = np.array(true_distances).reshape(-1, len(tags))
    visualization[tag_id] = {"img_id":img_id, "dist":true_distances}

IndexError: list index out of range

In [ ]:
for 